In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

In [2]:
z = torch.FloatTensor([1,2,3])
hypothesis = F.softmax(z, dim=0)
hypothesis

tensor([0.0900, 0.2447, 0.6652])

In [3]:
hypothesis.sum()

tensor(1.)

In [4]:
# cross entropy
z = torch.rand(3,5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
hypothesis

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)

In [5]:
y = torch.randint(5,(3,)).long()
y

tensor([0, 2, 1])

In [6]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [7]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
cost

tensor(1.4689, grad_fn=<MeanBackward0>)

In [8]:
torch.log(F.softmax(z,dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [9]:
F.log_softmax(z,dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [10]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [11]:
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

### Training with Low-level Cross Entropy Loss

In [13]:
X_train = [[1,2,1,1], 
           [2,1,3,2],
           [3,1,3,4],
           [4,1,5,5],
           [1,7,5,5],
           [1,2,5,6],
           [1,6,6,6],
           [1,7,7,7]]
y_train = [2,2,2,1,1,1,0,0]

X_train = torch.FloatTensor(X_train) # (8,4)
y_train = torch.LongTensor(y_train) # (8,)

In [14]:
y_train.shape

torch.Size([8])

In [16]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

epochs = 1000

for epoch in range(epochs+1):
    hypothesis = F.softmax(X_train.matmul(W)+b, dim=1)
    
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print('Epoch: {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))
    
    
    

Epoch:    0/1000 Cost: 1.098612
Epoch:  100/1000 Cost: 0.901535
Epoch:  200/1000 Cost: 0.839114
Epoch:  300/1000 Cost: 0.807826
Epoch:  400/1000 Cost: 0.788472
Epoch:  500/1000 Cost: 0.774822
Epoch:  600/1000 Cost: 0.764449
Epoch:  700/1000 Cost: 0.756191
Epoch:  800/1000 Cost: 0.749398
Epoch:  900/1000 Cost: 0.743671
Epoch: 1000/1000 Cost: 0.738749


### Training with F.cross_entropy

In [17]:
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W,b], lr=0.1)

epochs = 1000

for epoch in range(epochs+1):
    hypothesis = X_train.matmul(W) + b
    
    
    cost = F.cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print('Epoch: {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))
    

Epoch:    0/1000 Cost: 1.098612
Epoch:  100/1000 Cost: 0.761050
Epoch:  200/1000 Cost: 0.689991
Epoch:  300/1000 Cost: 0.643229
Epoch:  400/1000 Cost: 0.604117
Epoch:  500/1000 Cost: 0.568255
Epoch:  600/1000 Cost: 0.533922
Epoch:  700/1000 Cost: 0.500291
Epoch:  800/1000 Cost: 0.466908
Epoch:  900/1000 Cost: 0.433507
Epoch: 1000/1000 Cost: 0.399962


### High-level implementation witn nn.Module

In [21]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4,3) # dim = 4, classes = 3
        
    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()


optimizer = optim.SGD(model.parameters(), lr=0.1)

epochs = 1000

for epoch in range(epochs+1):
    hypothesis = model(X_train)
    
    
    cost = F.cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print('Epoch: {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))
        

Epoch:    0/1000 Cost: 2.778284
Epoch:  100/1000 Cost: 0.642843
Epoch:  200/1000 Cost: 0.558534
Epoch:  300/1000 Cost: 0.504019
Epoch:  400/1000 Cost: 0.459650
Epoch:  500/1000 Cost: 0.420177
Epoch:  600/1000 Cost: 0.383222
Epoch:  700/1000 Cost: 0.347261
Epoch:  800/1000 Cost: 0.311141
Epoch:  900/1000 Cost: 0.274520
Epoch: 1000/1000 Cost: 0.244428
